# Permutation engine examples

This shows how to use the permutation engine to permute gates of a certain type towards the beginning or end of the circuit.
It relies on the implementation of permutation rules between different gates. Currently, only rotational gates + Pauli + S + H + T and TimeEvolution gates are supported. So far there is no support for Controlled gates. But a CNOT can easily decomposed into two pi/4 rotations.

In [1]:
import projectq
import cmath
from projectq.backends import CommandPrinter, Simulator
from projectq.ops import *
from projectq.cengines import PermutePi4Front

#### First generate the main engine and the order of all the engines in the compiler stack.

In [2]:
engines = [CommandPrinter(),projectq.cengines.TagRemover(),PermutePi4Front()]
eng = projectq.MainEngine(backend=CommandPrinter(), engine_list=engines)

Now lets generate a very simple circuit

In [3]:
qubit = eng.allocate_qubit()
X | qubit
Y | qubit
X | qubit
T | qubit
T | qubit
eng.flush()

Allocate | Qureg[0]
X | Qureg[0]
Y | Qureg[0]
X | Qureg[0]
T | Qureg[0]
T | Qureg[0]
Allocate | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rx(3.14159265359) | Qureg[0]
Ry(3.14159265359) | Qureg[0]
Rx(3.14159265359) | Qureg[0]


Now lets take a look at two qubit gates. Such as the CNOT gate. For this we need to use the replacement rules provided by the projectq replacement engine. For this we first need to define which gates are allowed and which need to be replaced. This has been done in setups already.

In [4]:
from projectq.setups.surface_codes import lattice_surgery

In [5]:
engines2 = lattice_surgery.get_engine_list()
print(engines2)

[<projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fab6f841a20>, <projectq.cengines._tagremover.TagRemover object at 0x7fab6f85f7f0>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7fab6f85f828>, <projectq.cengines._optimize.LocalOptimizer object at 0x7fab6f85f860>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fab6f85f898>, <projectq.cengines._tagremover.TagRemover object at 0x7fab6f85f8d0>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7fab6f85f908>, <projectq.cengines._optimize.LocalOptimizer object at 0x7fab6f85f940>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fab6f85f978>, <projectq.cengines._tagremover.TagRemover object at 0x7fab6f85f9b0>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7fab6f85f9e8>, <projectq.cengines._optimize.LocalOptimizer object at 0x7fab6f85fa20>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fab6f85fa90>, <projectq.c

In [6]:
eng2 = projectq.MainEngine(backend=CommandPrinter(accept_input=False), engine_list=engines2, verbose=True)

qubit1 = eng2.allocate_qubit()
qubit2 = eng2.allocate_qubit()

H | qubit1
CNOT | (qubit1, qubit2)
eng2.flush()

Allocate | Qureg[0]
H | Qureg[0]
Allocate | Qureg[1]
exp(0.7853981633974483j * (1.0 Z0 X1)) | Qureg[0-1]
Rz(3.14159265359) | Qureg[0]
Rx(3.14159265359) | Qureg[1]
